## notebook only

In [1]:
# kedro
from kedro.framework.context import load_context
proj_path = '../../../' 
context = load_context(proj_path)
race_results_df_processed = catalog.load("race_results_df_processed")
parameters = context.params

# import from src
sys.path.append("./../../../src/forecast_keiba/")
from models.train_lightgbm import split_data

2020-09-21 10:46:56,589 - kedro.io.data_catalog - INFO - Loading data from `race_results_df_processed` (PickleDataSet)...


## import文
メイン関数実行のため

In [2]:
import pandas as pd
import pickle
import time
from tqdm.notebook import tqdm
import datetime as dt
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
import lightgbm as lgb

## メイン関数

In [3]:
def train_lightgbm(race_results_df_processed, parameters):
    print('start train_lr')
    race_results_df_processed.sort_values('date', ascending=False)
    df = race_results_df_processed[(race_results_df_processed['date'] > dt.datetime(2010,1,1)) & (race_results_df_processed['date'] < dt.datetime(2019,12,31))]
    df['rank'] = df['着順'].map(lambda x: 1 if x < 6 else (3 if x > 11 else 2))
    df.to_pickle('../../../data/processed/train_data.pickle')
    train,test = split_data(df,0.3)
    rank_1 = train['rank'].value_counts()[1]
    rank_2 = train['rank'].value_counts()[2]
    rank_3 = train['rank'].value_counts()[3]
    rus = RandomUnderSampler(sampling_strategy={1:rank_1,2:rank_2,3:rank_3},random_state=71)

    X_train = train.drop(['着順','date','rank'],axis=1)
    y_train = train['rank']
    X_test = test.drop(['着順','date','rank'],axis=1)
    y_test = test['rank']

    X_train_rus,y_train_rus = rus.fit_sample(X_train,y_train)
    
    params = {
    "num_leaves": 64,
    "n_estimators": 80,
    "class_weight": "balanced",
    "random_state": 100,
    "max_depth": 24,
    }

    model_lightgbm = lgb.LGBMClassifier(**params)
    model_lightgbm.fit(X_train_rus.values,y_train_rus.values)

    print('train score: ' + str(model_lightgbm.score(X_train,y_train)))
    print('test score: ' + str(model_lightgbm.score(X_test,y_test)))
    
    print('feature importances')
    importances = pd.DataFrame(
    {"features": X_train.columns, "importance": model_lightgbm.feature_importances_}
    )
    print(importances.sort_values("importance", ascending=False)[:20])
    pickle.dump(model_lightgbm, open('../../../data/models/model_lightgbm', 'wb'))
    
    return model_lightgbm

In [4]:
def main(race_results_df_processed, parameters):
    return train_lightgbm(race_results_df_processed, parameters)

if __name__ == "__main__":
    main(race_results_df_processed, parameters)

start train_lr
train score: 0.2903225806451613
test score: 0.3125
feature importances
   features  importance
0        枠番           0
58  peds_41           0
67  peds_50           0
66  peds_49           0
65  peds_48           0
64  peds_47           0
63  peds_46           0
62  peds_45           0
61  peds_44           0
60  peds_43           0
59  peds_42           0
57  peds_40           0
1        馬番           0
56  peds_39           0
55  peds_38           0
54  peds_37           0
53  peds_36           0
52  peds_35           0
51  peds_34           0
50  peds_33           0
